**Import libraries**

In [2]:
import math
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

**Default parameters**

In [34]:
MAX_SIZE = 783
NUM_WORDS = 1000
EMBEDDING_DIM = 16
EPOCHS = 10
BATCH_SIZE = 128
OOV = 0
SARCASM_TRAINING_SIZE = 20000

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Creating DataFrames**

In [5]:
goodreads_train = pd.read_csv("kaggle/input/goodreads_train.csv")
goodreads_test = pd.read_csv("kaggle/input/goodreads_test.csv")

In [6]:
goodreads_train.sample(5)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
620942,ecbfc359e8b4a7d05f549e30f489c501,29405105,e7c6afdf00497a24aa129a55e1545cfd,5,5 stars \n I am always excited when the latest...,Sun Jul 05 23:07:06 -0700 2015,Fri Oct 27 04:12:19 -0700 2017,Tue May 24 21:35:03 -0700 2016,NaN,0,0
196121,848fb2c7399065f74f24f2ec15a78759,10369932,cca3aa581774f49f6f8a6f63f7b85fc7,3,Perfect for a middle school audience; my book ...,Sun Jun 25 18:53:45 -0700 2017,Tue Jul 11 06:58:43 -0700 2017,Tue Jul 11 06:58:43 -0700 2017,Sun Jul 09 09:48:40 -0700 2017,0,0
716590,9f7d458360179a07a6c092778676d651,22826126,6957d81fd3cb0dd1ff085199488e1e40,4,"Really a 4.5 star review. (Basically, 5 stars ...",Thu Sep 07 20:58:02 -0700 2017,Thu Sep 07 21:13:04 -0700 2017,Wed Sep 06 00:00:00 -0700 2017,Mon Sep 04 00:00:00 -0700 2017,1,2
383182,d8d4cad6c612f47c781143d35255d497,16181775,cfcd7cb1b687ecd6a2508ef5f7b30560,5,Good Story #82. Julie and Scott discover that ...,Wed Jan 29 13:55:39 -0800 2014,Sat Apr 29 07:54:26 -0700 2017,Sat Apr 29 07:54:26 -0700 2017,Tue Apr 25 00:00:00 -0700 2017,5,2
737901,e5e161374b0acd1c063c99508546244b,16101128,04787f4d58130b0f1473e9c7adfaf5a8,3,I believe I'm a bit too old for young adult bo...,Wed Jul 20 10:36:16 -0700 2016,Wed Jul 20 10:59:08 -0700 2016,Sat Jul 16 00:00:00 -0700 2016,NaN,0,0


In [7]:
goodreads_test.sample(5)

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
201994,50e2ef79c502ae6e28c056eb1c9f9d89,22545870,3d7df4ff2abb0235af622a167f819a1f,"If I hadn't known any better, I would have ass...",Sun Apr 05 16:06:15 -0700 2015,Mon Jun 22 10:05:14 -0700 2015,Mon Apr 06 15:33:25 -0700 2015,Sun Apr 05 00:00:00 -0700 2015,2,0
398832,501209dbf1919c8bb0a32351385fc883,9938498,149d776a1dee7b4ccf9f604e5ffe544a,I read this as a buddy read with my Goodreads'...,Thu Mar 31 16:28:51 -0700 2011,Mon Aug 22 03:34:38 -0700 2011,Mon Aug 22 00:00:00 -0700 2011,Fri Aug 12 00:00:00 -0700 2011,6,39
152805,1b790a9d988cceacf12b08e6ccc7c2f8,8667848,141d55bb62a7a08f14fd63eb6b431055,"All in all. Started really interesting, and th...",Mon Jun 13 14:36:58 -0700 2011,Mon Jun 20 07:00:57 -0700 2011,Tue Feb 01 00:00:00 -0800 2011,NaN,0,0
445181,ab89d10cc2d51c1eeb4d5d5077ed7fb9,16096824,3e1b68aafcf725a4bd9ad48ca328efb1,**RANTISH** \n This book left me with a bunch ...,Tue Aug 18 02:00:44 -0700 2015,Mon Aug 24 22:01:54 -0700 2015,Sun Aug 23 00:00:00 -0700 2015,Tue Aug 18 00:00:00 -0700 2015,0,0
399345,932971ffd04486a4a3a2c040c7ca32c3,29772863,b3a50190db53c3cb70dc47e1c590cd23,"That was adorably epic!!! I loved the art, sto...",Fri Jan 13 10:17:07 -0800 2017,Sun Feb 05 07:47:18 -0800 2017,Sun Feb 05 07:47:18 -0800 2017,Sat Feb 04 00:00:00 -0800 2017,0,0


**Cleaning Data**

In [8]:
train_df = goodreads_train.drop(columns=['user_id', 'book_id', 'date_added', 'date_updated', 'read_at', 'started_at'],
                                axis=0)
train_df.head()

,review_id,rating,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",9,1


In [9]:
train_df.dtypes

review_id      object
rating          int64
review_text    object
n_votes         int64
n_comments      int64
dtype: object

In [10]:
y_train = train_df['rating']
x_train = train_df.drop('rating', axis=1)

In [11]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,"I really enjoyed this book, and there is a lot...",9,1


In [12]:
y_train.head()

0    5
1    3
2    3
3    0
4    4
Name: rating, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [14]:
x_train['review_id'] = le.fit_transform(x_train['review_id'])

In [15]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,786842,This is a special book. It started slow for ab...,28,1
1,583423,Recommended by Don Katz. Avail for free in Dec...,1,0
2,165147,"A fun, fast paced science fiction thriller. I ...",22,0
3,727692,Recommended reading to understand what is goin...,5,1
4,179941,"I really enjoyed this book, and there is a lot...",9,1


**NLP**

In [16]:
def get_sequences(tokenizer, review):
    sequences = tokenizer.texts_to_sequences(review)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=MAX_SIZE, padding='post')
    return padded_sequences

In [17]:
def tokenizer_func(data_rating, data_review):
    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV)
    tokenizer.fit_on_texts(data_review)

    """word_index = tokenizer.word_index"""

    train_labels = data_rating.iloc[math.floor(int(len(data_rating) / 8)):]
    train_examples = data_review.iloc[math.floor(int(len(data_review) / 8)):]
    test_examples = data_review.iloc[:math.floor(int(len(data_review) / 8))]
    test_labels = data_rating.iloc[:math.floor(int(len(data_rating) / 8))]

    padded_train = get_sequences(tokenizer, train_examples)
    padded_test = get_sequences(tokenizer, test_examples)

    return np.array(padded_train), np.array(padded_test), np.array(train_labels), np.array(test_labels)

In [18]:
padded_train, padded_test, train_labels, test_labels = tokenizer_func(y_train, x_train['review_text'])

In [19]:
padded_train[0]

array([  4,  12, 294,  16,   6,   1, 566,  31,   3,   9,  10, 456,  42,
         4, 151,  88, 597,   1,  59,  10,   6,   1,   7,   6,   1,   1,
        21,  10, 635,  16,   2,   1,   1, 459,  45,  34, 185,   1,  27,
         6, 428,   5,   1,   2, 223,   7,   2, 888,   1,   1, 388,  42,
        80,   4, 134,  47,   1, 362,   3, 566,  32,  42,   4,  12, 294,
        16,   4,  91,  37, 291,  13,  14,  15,   9, 167, 209, 124, 207,
        25, 132,  48, 156, 290,   3,  84,   1, 255, 111, 383, 242, 124,
         3,  42, 180,   7, 460,  10,   1,   1,  24,   2, 434,  70, 914,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

**Sarcasm detection**

In [20]:
def define_sarcasm():
    data = pd.read_json('kaggle/input/sarcasm.json', lines=True)
    # iterating through the json data and loading the requisite values into our python lists
    sentences = data['headline']
    labels = data['is_sarcastic']

    training_sentences = sentences[0:SARCASM_TRAINING_SIZE]
    testing_sentences = sentences[SARCASM_TRAINING_SIZE:]

    training_labels = labels[0:SARCASM_TRAINING_SIZE]
    testing_labels = labels[SARCASM_TRAINING_SIZE:]
    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV)
    # fitting tokenizer only to training set
    tokenizer.fit_on_texts(training_sentences)

    word_index = tokenizer.word_index

    # creating training sequences and padding them
    traning_sequences = tokenizer.texts_to_sequences(training_sentences)
    training_padded = pad_sequences(traning_sequences, maxlen=MAX_SIZE,
                                    padding='post',
                                    truncating='post',
                                    )

    # creating  testing sequences and padding them using same tokenizer
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
    testing_padded = pad_sequences(testing_sequences, maxlen=MAX_SIZE,
                                   padding='post',
                                   truncating='post',
                                   )

    # converting all variables to numpy arrays, to be able to work with tf version 2
    training_padded = np.array(training_padded)
    training_labels = np.array(training_labels)
    testing_padded = np.array(testing_padded)
    testing_labels = np.array(testing_labels)

    # creating a model for sentiment analysis
    model = tf.keras.Sequential([
        # addinging an Embedding layer for Neural Network to learn the vectors
        tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SIZE),
        # Global Average pooling is similar to adding up vectors in this case
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    """model.fit(training_padded, training_labels, epochs=EPOCHS,
                        validation_data=(testing_padded, testing_labels))"""

    return model

In [21]:
sarcasm_model = define_sarcasm()

In [22]:
sarcasm_prediction_train = sarcasm_model.predict(padded_train)
sarcasm_prediction_test = sarcasm_model.predict(padded_test)

3516/3516 [==============================] - 8s 2ms/step


**Reshaping data**


In [23]:
print(sarcasm_prediction_train.shape)

(787500, 1)


In [24]:
print(padded_train.shape)

(787500, 783)


In [25]:
padded_train = np.concatenate((padded_train, np.array(sarcasm_prediction_train.flatten())[:, None]), axis=1)
padded_test = np.concatenate((padded_test, np.array(sarcasm_prediction_test.flatten())[:, None]), axis=1)

In [26]:
print(padded_train.shape)

(787500, 784)


In [27]:
padded_train = np.reshape(padded_train, (1 - math.floor(len(y_train) / 8), 28, 28))
padded_test = np.reshape(padded_test, (math.floor(int(len(x_train['review_text']) / 8)), 28, 28))

In [28]:
print(padded_train[0])

[[4.000000e+00 1.200000e+01 2.940000e+02 1.600000e+01 6.000000e+00
  1.000000e+00 5.660000e+02 3.100000e+01 3.000000e+00 9.000000e+00
  1.000000e+01 4.560000e+02 4.200000e+01 4.000000e+00 1.510000e+02
  8.800000e+01 5.970000e+02 1.000000e+00 5.900000e+01 1.000000e+01
  6.000000e+00 1.000000e+00 7.000000e+00 6.000000e+00 1.000000e+00
  1.000000e+00 2.100000e+01 1.000000e+01]
 [6.350000e+02 1.600000e+01 2.000000e+00 1.000000e+00 1.000000e+00
  4.590000e+02 4.500000e+01 3.400000e+01 1.850000e+02 1.000000e+00
  2.700000e+01 6.000000e+00 4.280000e+02 5.000000e+00 1.000000e+00
  2.000000e+00 2.230000e+02 7.000000e+00 2.000000e+00 8.880000e+02
  1.000000e+00 1.000000e+00 3.880000e+02 4.200000e+01 8.000000e+01
  4.000000e+00 1.340000e+02 4.700000e+01]
 [1.000000e+00 3.620000e+02 3.000000e+00 5.660000e+02 3.200000e+01
  4.200000e+01 4.000000e+00 1.200000e+01 2.940000e+02 1.600000e+01
  4.000000e+00 9.100000e+01 3.700000e+01 2.910000e+02 1.300000e+01
  1.400000e+01 1.500000e+01 9.000000e+00 1.67

**CNN Training**

In [55]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
model.add(tf.keras.layers.Dense(6,
                                activation=tf.keras.activations.softmax))  # model.add(tf.keras.layers.Dense(1, activation=tf.keras.activations.softmax))

In [56]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.1, momentum=0.9),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [57]:
model.fit(padded_train,
          train_labels,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS
          )

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

**Testing model**

In [ ]:
def model_start(padded_train, padded_test, train_labels, test_labels, model):
    padded_train = padded_train / 255.0
    padded_test = padded_test / 255.0

    train_labels = tf.keras.utils.to_categorical(train_labels, 6)
    test_labels = tf.keras.utils.to_categorical(test_labels, 6)

    padded_train = np.expand_dims(padded_train, -1)
    padded_test = np.expand_dims(padded_test, -1)

    model.predict(padded_train)
    model.summary()

    return model

**Model run**

In [ ]:
model = model_start(padded_train, padded_test, train_labels, test_labels, model)